# GameDay2021 Live GUI Draft Example

See the GameDay_Notebook for examples of running simulations.  This page will focus in-depth on using the code in a live-draft situation.  

You need three components, all which are provided, but can be edited/substituted:
1. Projections
2. Rankings
3. Injuries

#### Projections 
Made available for you via [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips) are ZiPS, Steamer, and TheBat projections.  Adding your own shouldn't be too hard, see the directories and data for formatting.  
#### Rankings 
Available rankings were scraped from FantasyPros, which give you the option to choose from Yahoo, ESPN, etc., so that adding your own is simple and could provide an edge.  Case in point, I'm working on my own ranking based off simulations and will soon add the option.  
#### Injuries
Filtering out players no longer available will improve the prediction power of the code.  Simply add names to the injury spreadsheet provided.  
#### Live-Draft Picks
New GUI replaces previous editing of Excel spreadsheet but still requires user input.  Since drafting is live, you will need to enter the draft results of every team as it's happening.  


In [1]:
import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import copy
from PyQt5 import QtCore, QtGui, QtWidgets

from GameDayFunctions.fangraphs_projection_2021 import Projection
from GameDayFunctions.draft_2021 import Draft

In [2]:
autodraft_depth = 'end'
search_depth = 2
year = 2021
path_data = "projections/"
path_drafts = "Draft_Pick_Spreadsheets/"
shuffle_picks = False
ranking_method = 'FantasyPros'
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS',ranking_method = ranking_method)
player_rankings = {"ZiPS":player_ranking_zips}
number_teams = 12
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':1}
batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] 
draft_position = 8

# Choose the projection system you prefer ()
zips = Draft(player_rankings['ZiPS'], draft_position = draft_position, number_teams = number_teams,roster_spots = roster_spots,batter_stats = batter_stats,pitcher_stats = pitcher_stats)

projections/2021/ZiPS_2021_Pitchers.csv
projections/2021/ZiPS_2021_Hitters.csv


In [3]:
import pdb
import os
import sys
import pandas as pd
from PyQt5.QtWidgets import QApplication, QWidget, QTableWidget, QTableWidgetItem, QHeaderView, QLineEdit, \
                            QPushButton, QItemDelegate, QVBoxLayout
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QDoubleValidator

class FloatDelegate(QItemDelegate):
    def __init__(self, parent=None):
        super().__init__()

    def createEditor(self, parent, option, index):
        editor = QLineEdit(parent)
        editor.setValidator(QDoubleValidator())
        return editor

class TableWidget(QTableWidget):
    def __init__(self, df):
        super().__init__()
        self.df = df
        self.setStyleSheet('font-size: 16px;')

        # set table dimension
        nRows, nColumns = self.df.shape
        self.setColumnCount(nColumns)
        self.setRowCount(nRows)

        self.setHorizontalHeaderLabels(('Picks', 'PLAYER', 'EligiblePosition'))
        self.verticalHeader().setSectionResizeMode(QHeaderView.Stretch)
        self.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)

        self.setItemDelegateForColumn(1, FloatDelegate())

        # data insertion
        for i in range(self.rowCount()):
            for j in range(self.columnCount()):
                self.setItem(i, j, QTableWidgetItem(str(self.df.iloc[i, j])))

        self.cellChanged[int, int].connect(self.updateDF)   

    def updateDF(self, row, column):
        text = self.item(row, column).text()
        self.df.iloc[row, column] = text
        
class DFEditor(QWidget):

    #path = "/Users/marcoviero/Code/Git_Repositories/GameDay2021/projections/2021/PositionalRankings/ESPN/ESPN_Roto_Ranking_2021.xlsx"
    #df = pd.read_excel(path)
    #df['Pick'] = ""
    #df = df[['Pick', 'PLAYER', 'Elig. Pos.']]
    
    def __init__(self, df, gameday_df, draft_pick_file = 'Draft_pick_export.xlsx'):
        super().__init__()
        self.resize(900, 600)
        
        self.draft_pick_file = draft_pick_file
        self.gameday_df = gameday_df

        mainLayout = QVBoxLayout()

        self.table = TableWidget(df)
        mainLayout.addWidget(self.table)

        #button_print = QPushButton('Sort Picks')
        #button_print.setStyleSheet('font-size: 14px')
        #button_print.clicked.connect(self.display_sorted_picks)
        #mainLayout.addWidget(button_print)
        
        button_print = QPushButton('Draft Player')
        button_print.setStyleSheet('font-size: 14px')
        button_print.clicked.connect(self.make_draft_pick)
        mainLayout.addWidget(button_print)

        button_export = QPushButton('Export to CSV file')
        button_export.setStyleSheet('font-size: 14px')
        button_export.clicked.connect(self.export_to_csv)
        mainLayout.addWidget(button_export)     

        self.setLayout(mainLayout)
        
    def make_draft_pick(self):
        self.export_to_csv()
        autodraft_depth = 'end'
        draft_pick_file = self.draft_pick_file
        player_to_pick, projected_roto_stats = self.gameday_df.draft_from_list_and_find_best_pick(search_depth = 2, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)
    
    def display_sorted_picks(self):
        #print(self.table.df)
        self.table.df["Pick"][self.table.df.Pick == ""] = "9999"
        self.table.df = self.table.df.sort_values(by='Pick')
        self.table.df["Pick"][self.table.df.Pick == "9999"] = ""
        

    def export_to_csv(self):
        df_out = self.table.df.copy()
        df_out["Pick"][df_out.Pick == ""] = "9999"
        df_out = df_out.sort_values(by='Pick')
        df_out = df_out[df_out.Pick != "9999"] 
        df_out.to_excel(os.path.join('Draft_Pick_Spreadsheets',self.draft_pick_file), index=False)
        print('CSV file exported.')

#if __name__ == '__main__':
#    app = QApplication(sys.argv)
#    path = "/Users/marcoviero/Code/Git_Repositories/GameDay2021/projections/2021/PositionalRankings/ESPN/ESPN_Roto_Ranking_2021.xlsx"
#    df = pd.read_excel(path)
#    df['Pick'] = ""
#    df = df[['Pick', 'PLAYER', 'Elig. Pos.']]
#    demo = DFEditor(df)
#    demo.show()
#    
#    sys.exit(app.exec_())

In [4]:
def call_gameday2021_gui(gameday_df, path="/Users/marcoviero/Code/Git_Repositories/GameDay2021/projections/2021/PositionalRankings/",ranking='ESPN'):
    filename = ranking+'_Roto_Ranking_2021.xlsx'
    filepath = os.path.join(path,ranking,filename) 
    
    app = QApplication(sys.argv)
    df = pd.read_excel(filepath)
    df['Pick'] = ""
    df = df[['Pick', 'PLAYER', 'Elig. Pos.']]
    demo = DFEditor(df, gameday_df)
    demo.show()

    sys.exit(app.exec_())

In [ ]:
call_gameday2021_gui(zips, ranking='Yahoo')

CSV file exported.
Finding Best Pick For Team 8
Stored Result for Pick 0 [8/7] Trea Turner SS whose placement/score is 7/127.5
Stored Result for Pick 1 [8/9] Christian Yelich OF whose placement/score is 8/127.5
Stored Result for Pick 2 [8/10] Trevor Story SS whose placement/score is 6/142.0
Stored Result for Pick 3 [8/11] Shane Bieber SP whose placement/score is 3/142.0
Stored Result for Pick 4 [8/12] Jose Ramirez 3B whose placement/score is 5/136.5
Stored Result for Pick 5 [8/13] Freddie Freeman 1B whose placement/score is 6/133.5
Stored Result for Pick 6 [8/14] Bryce Harper OF whose placement/score is 2/153.5
Stored Result for Pick 7 [8/15] Cody Bellinger 1B whose placement/score is 5/143.0
Stored Result for Pick 8 [8/17] Manny Machado 3B whose placement/score is 5/137.0
Stored Result for Pick 9 [8/18] Trevor Bauer SP whose placement/score is 10/120.5
Stored Result for Pick 10 [8/32] Ozzie Albies 2B whose placement/score is 9/121.0
Stored Result for Pick 11 [8/34] DJ LeMahieu 2B whos

(Pdb)  c


> /Users/marcoviero/Code/Git_Repositories/GameDay2021/GameDayFunctions/draft_2021.py(485)draft_next_best()
-> teams[team_key] = self.draft_into_teams(teams[team_key], drafted_player, silent = True)


(Pdb)  c


UnboundLocalError: local variable 'drafted_player' referenced before assignment

...

CSV file exported.
Finding Best Pick For Team 6
> /Users/marcoviero/Code/Git_Repositories/GameDay2021/GameDayFunctions/draft_2021.py(275)draft_remaining()
-> teams_copy, df_copy = self.draft_next_best(iteam, teams_copy, df_copy, iround, shuffle_picks = shuffle_picks)


(Pdb)  c


> /Users/marcoviero/Code/Git_Repositories/GameDay2021/GameDayFunctions/draft_2021.py(485)draft_next_best()
-> teams[team_key] = self.draft_into_teams(teams[team_key], drafted_player, silent = True)


(Pdb)  c


UnboundLocalError: local variable 'drafted_player' referenced before assignment

In [1]:
import pdb
import sys
import pandas as pd
import numpy as np
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import Qt, QSortFilterProxyModel

class TableModel(QtCore.QAbstractTableModel):
    def __init__(self, data):
        super(TableModel, self).__init__()
        self._data = data

    def data(self, index, role):
        if role == Qt.DisplayRole:
            # See below for the nested-list data structure.
            # .row() indexes into the outer list,
            # .column() indexes into the sub-list
            pdb.set_trace()
            return self._data[index.row()][index.column()]

    def rowCount(self, index):
        # The length of the outer list.
        return len(self._data)

    def columnCount(self, index):
        # The following takes the first sub-list, and returns
        # the length (only works if all rows are an equal length)
        return len(self._data[0])

class DataFrameModel(QtCore.QAbstractTableModel):
    def __init__(self, df):
        super(DataFrameModel, self).__init__()
        self._df = df
        pdb.set_trace()
        #self.cellChanged[int, int].connect(self.updateDF)   

    def data(self, index, role):
        if role == Qt.DisplayRole:
            # See below for the nested-list data structure.
            # .row() indexes into the outer list,
            # .column() indexes into the sub-list
            return self._df.iloc[index.row()][index.column()]

    def rowCount(self, index):
        # The length of the outer list.
        return np.shape(self._df)[0]

    def columnCount(self, index):
        # The following takes the first sub-list, and returns
        # the length (only works if all rows are an equal length)
        return np.shape(self._df)[1]   
    
    def updateDF(self, row, column):
        text = self.item(row, column).text()
        self.df.iloc[row, column] = text
        #pdb.set_trace()
        self.df = self.df.sort_values(by='Pick')[::-1]
    
    def flags(self, index):
        if (index.column() == 0):
            return QtCore.Qt.ItemIsEditable | QtCore.Qt.ItemIsEnabled
        else:
            return QtCore.Qt.ItemIsEnabled 
        
class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.resize(900, 600)

        self.table = QtWidgets.QTableView()

        path = "/Users/marcoviero/Code/Git_Repositories/GameDay2021/projections/2021/PositionalRankings/ESPN/ESPN_Roto_Ranking_2021.xlsx"
        df = pd.read_excel(path)
        df['Pick'] = ""
        df = df[['Pick', 'PLAYER', 'Elig. Pos.']]
        
        self.model = DataFrameModel(df)
        #self.table.setModel(self.model)
        self.proxyModel = QSortFilterProxyModel()
        self.proxyModel.setSourceModel(self.model)

        self.table.setSortingEnabled(True)

        self.table.setModel(self.proxyModel)
        self.setCentralWidget(self.table)


app=QtWidgets.QApplication(sys.argv)
window=MainWindow()
window.show()
app.exec_()

--Return--
> <ipython-input-1-a008ca4f97a3>(34)__init__()->None
-> pdb.set_trace()


(Pdb)  c


0

In [1]:
import pdb
import sys
import pandas as pd
from PyQt5.QtWidgets import QApplication, QWidget, QTableWidget, QTableView, QTableWidgetItem, QHeaderView, QLineEdit, \
                            QPushButton, QItemDelegate, QVBoxLayout
from PyQt5.QtCore import Qt, QSortFilterProxyModel, QAbstractTableModel
from PyQt5.QtGui import QDoubleValidator

class FloatDelegate(QItemDelegate):
    def __init__(self, parent=None):
        super().__init__()

    def createEditor(self, parent, option, index):
        editor = QLineEdit(parent)
        editor.setValidator(QDoubleValidator())
        return editor

class TableWidget(QTableWidget):
    def __init__(self, df):
        super().__init__()
        self.df = df
        self.setStyleSheet('font-size: 18px;')

        # set table dimension
        nRows, nColumns = self.df.shape
        self.setColumnCount(nColumns)
        self.setRowCount(nRows)

        self.setHorizontalHeaderLabels(('Pick', 'Player', 'Elig. Pos.'))
        self.verticalHeader().setSectionResizeMode(QHeaderView.Stretch)
        self.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)

        self.setItemDelegateForColumn(1, FloatDelegate())

        # data insertion
        for i in range(self.rowCount()):
            for j in range(self.columnCount()):
                self.setItem(i, j, QTableWidgetItem(str(self.df.iloc[i, j])))

        self.cellChanged[int, int].connect(self.updateDF)   

    def updateDF(self, row, column):
        text = self.item(row, column).text()
        self.df.iloc[row, column] = text
        #pdb.set_trace()
        self.df = self.df.sort_values(by='Pick')[::-1]
    
    def flags(self, index):
        if (index.column() == 0):
            return QtCore.Qt.ItemIsEditable | QtCore.Qt.ItemIsEnabled
        else:
            return QtCore.Qt.ItemIsEnabled 
        
class TableModel(QAbstractTableModel):
    def __init__(self, data):
        super(TableModel, self).__init__()
        self._data = data

    def data(self, index, role):
        if role == DisplayRole:
            # See below for the nested-list data structure.
            # .row() indexes into the outer list,
            # .column() indexes into the sub-list
            return self._data[index.row()][index.column()]

    def rowCount(self, index):
        # The length of the outer list.
        return len(self._data)

    def columnCount(self, index):
        # The following takes the first sub-list, and returns
        # the length (only works if all rows are an equal length)
        return len(self._data[0])
    
class DFEditor(QWidget):

    path = "/Users/marcoviero/Code/Git_Repositories/GameDay2021/projections/2021/PositionalRankings/ESPN/ESPN_Roto_Ranking_2021.xlsx"
    df = pd.read_excel(path)
    df['Pick'] = ""
    df = df[['Pick', 'PLAYER', 'Elig. Pos.']]
        
    def __init__(self):
        super().__init__()
        self.resize(900, 600)

        mainLayout = QVBoxLayout()

        #self.table = TableWidget(DFEditor.df)
        #mainLayout.addWidget(self.table)

        self.model = TableModel(DFEditor.df)
        self.table = QTableView()

        self.proxyModel = QSortFilterProxyModel()
        self.proxyModel.setSourceModel(self.model)

        self.table.setSortingEnabled(True)

        self.table.setModel(self.proxyModel)
        mainLayout.addWidget(self.proxyModel)
    
        button_print = QPushButton('Sort Picks')
        button_print.setStyleSheet('font-size: 15px')
        button_print.clicked.connect(self.display_sorted_picks)
        mainLayout.addWidget(button_print)

        button_export = QPushButton('Export to CSV file')
        button_export.setStyleSheet('font-size: 15px')
        button_export.clicked.connect(self.export_to_csv)
        mainLayout.addWidget(button_export)     

        self.setLayout(mainLayout)
        
    def display_sorted_picks(self):
        # set table dimension
        nRows, nColumns = self.df.shape
        self.setColumnCount(nColumns)
        self.setRowCount(nRows)
        for i in range(self.rowCount()):
            for j in range(self.columnCount()):
                self.setItem(i, j, QTableWidgetItem(str(self.df.iloc[i, j])))
                
    def export_to_csv(self):
        self.table.df.to_csv('Draft_pick_export.csv', index=False)
        print('CSV file exported.')

if __name__ == '__main__':
    app = QApplication(sys.argv)

    demo = DFEditor()
    demo.show()
    
    sys.exit(app.exec_())

KeyError: 0

TypeError: addWidget(self, QWidget, stretch: int = 0, alignment: Union[Qt.Alignment, Qt.AlignmentFlag] = Qt.Alignment()): argument 1 has unexpected type 'QSortFilterProxyModel'

In [1]:
class TableModel(QtCore.QAbstractTableModel): 
    def __init__(self, parent=None, *args): 
        super(TableModel, self).__init__()
        self.datatable = None

    def update(self, dataIn):
        print ('Updating Model')
        self.datatable = dataIn
        print ('Datatable : {0}'.format(self.datatable))

    def rowCount(self, parent=QtCore.QModelIndex()):
        return len(self.datatable.index) 

    def columnCount(self, parent=QtCore.QModelIndex()):
        return len(self.datatable.columns.values) 

    def data(self, index, role=QtCore.Qt.DisplayRole):
        if role == QtCore.Qt.DisplayRole:
            i = index.row()
            j = index.column()
            return '{0}'.format(self.datatable.iget_value(i, j))
        else:
            return QtCore.QVariant()

    def flags(self, index):
        return QtCore.Qt.ItemIsEnabled

NameError: name 'QtCore' is not defined

In [3]:
class DataFrameModel(QtCore.QAbstractTableModel):
    
    DtypeRole = QtCore.Qt.UserRole + 1000
    ValueRole = QtCore.Qt.UserRole + 1001

    def __init__(self, df=pd.DataFrame(), parent=None):
        super(DataFrameModel, self).__init__(parent)
        #QtCore.QAbstractTableModel.flags(self, index) | 
        #QtCore.Qt.ItemIsEditable
        self._dataframe = df

    def setDataFrame(self, dataframe):
        self.beginResetModel()
        self._dataframe = dataframe.copy()
        self.endResetModel()

    def dataFrame(self):
        return self._dataframe

    dataFrame = QtCore.pyqtProperty(pd.DataFrame, fget=dataFrame, fset=setDataFrame)

    @QtCore.pyqtSlot(int, QtCore.Qt.Orientation, result=str)
    def headerData(self, section: int, orientation: QtCore.Qt.Orientation, role: int = QtCore.Qt.DisplayRole):
        if role == QtCore.Qt.DisplayRole:
            if orientation == QtCore.Qt.Horizontal:
                return self._dataframe.columns[section]
            else:
                return str(self._dataframe.index[section])
        return QtCore.QVariant()

    def rowCount(self, parent=QtCore.QModelIndex()):
        if parent.isValid():
            return 0
        return len(self._dataframe.index)

    def columnCount(self, parent=QtCore.QModelIndex()):
        if parent.isValid():
            return 0
        return self._dataframe.columns.size

    def data(self, index, role=QtCore.Qt.DisplayRole):
        if not index.isValid() or not (0 <= index.row() < self.rowCount() \
            and 0 <= index.column() < self.columnCount()):
            return QtCore.QVariant()
        row = self._dataframe.index[index.row()]
        col = self._dataframe.columns[index.column()]
        dt = self._dataframe[col].dtype

        val = self._dataframe.iloc[row][col]
        if role == QtCore.Qt.DisplayRole:
            return str(val)
        elif role == DataFrameModel.ValueRole:
            return val
        if role == DataFrameModel.DtypeRole:
            return dt
        return QtCore.QVariant()

    def roleNames(self):
        roles = {
            QtCore.Qt.DisplayRole: b'display',
            DataFrameModel.DtypeRole: b'dtype',
            DataFrameModel.ValueRole: b'value'
        }
        return roles
    
    def flags(self, index):
        if (index.column() == 0):
            return QtCore.Qt.ItemIsEditable | QtCore.Qt.ItemIsEnabled
        else:
            return QtCore.Qt.ItemIsEnabled 

In [2]:
from PyQt5 import QtCore, QtGui, QtWidgets
import pandas as pd
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(662, 512)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.horizontalLayout = QtWidgets.QHBoxLayout(self.centralwidget)
        self.horizontalLayout.setObjectName("horizontalLayout")
        self.verticalLayout = QtWidgets.QVBoxLayout()
        self.verticalLayout.setObjectName("verticalLayout")
        #self.lineEdit = QtWidgets.QLineEdit(self.centralwidget)
        #self.lineEdit.setObjectName("lineEdit")
        #self.lineEdit = QtWidgets.QLineEdit("/Users/marcoviero/Code/Git_Repositories/GameDay2021/projections/2021/PositionalRankings/ESPN/ESPN_Roto_Ranking_2021.xlsx")
        #self.lineEdit.selectAll()
        #self.verticalLayout.addWidget(self.lineEdit)
        #self.tableView = QtWidgets.QTableWidgetItem(self.centralwidget)
        self.tableView = QtWidgets.QTableView(self.centralwidget)
        self.tableView.setObjectName("tableView")
        self.verticalLayout.addWidget(self.tableView)
        #self.tableView.setFlags(QtCore.Qt.ItemIsEditable | QtCore.Qt.ItemIsEnabled | QtCore.Qt.ItemIsSelectable)
        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setObjectName("pushButton")
        self.verticalLayout.addWidget(self.pushButton)
        self.horizontalLayout.addLayout(self.verticalLayout)
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 662, 21))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
        
        QtCore.Qt.ItemIsEditable | QtCore.Qt.ItemIsEnabled | QtCore.Qt.ItemIsSelectable

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.pushButton.setText(_translate("MainWindow", "PushButton"))
        
        self.lineEdit = QtWidgets.QLineEdit("/Users/marcoviero/Code/Git_Repositories/GameDay2021/projections/2021/PositionalRankings/ESPN/ESPN_Roto_Ranking_2021.xlsx")
        self.lineEdit.selectAll()

        self.pushButton.clicked.connect(self.btn_clk)

        MainWindow.show()

    def btn_clk(self):
        path = self.lineEdit.text()
        df = pd.read_excel(path)
        df['Pick'] = ""
        df2 = df[['Pick', 'PLAYER', 'Elig. Pos.']]
        self.model = DataFrameModel(df2)
        #self.model = TableModel(df2)
        self.tableView.setModel(self.model)

        

In [3]:
def launch_db_viewer():
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

In [4]:
launch_db_viewer()

NameError: name 'DataFrameModel' is not defined

SystemExit: 0

/Users/marcoviero/opt/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Backup 1

In [ ]:
class DataFrameModel(QtCore.QAbstractTableModel):
    DtypeRole = QtCore.Qt.UserRole + 1000
    ValueRole = QtCore.Qt.UserRole + 1001
    QtCore.Qt.ItemIsEditable

    def __init__(self, df=pd.DataFrame(), parent=None):
        super(DataFrameModel, self).__init__(parent)
        self._dataframe = df

    def setDataFrame(self, dataframe):
        self.beginResetModel()
        self._dataframe = dataframe.copy()
        self.endResetModel()

    def dataFrame(self):
        return self._dataframe

    dataFrame = QtCore.pyqtProperty(pd.DataFrame, fget=dataFrame, fset=setDataFrame)

    @QtCore.pyqtSlot(int, QtCore.Qt.Orientation, result=str)
    def headerData(self, section: int, orientation: QtCore.Qt.Orientation, role: int = QtCore.Qt.DisplayRole):
        if role == QtCore.Qt.DisplayRole:
            if orientation == QtCore.Qt.Horizontal:
                return self._dataframe.columns[section]
            else:
                return str(self._dataframe.index[section])
        return QtCore.QVariant()

    def rowCount(self, parent=QtCore.QModelIndex()):
        if parent.isValid():
            return 0
        return len(self._dataframe.index)

    def columnCount(self, parent=QtCore.QModelIndex()):
        if parent.isValid():
            return 0
        return self._dataframe.columns.size

    def data(self, index, role=QtCore.Qt.DisplayRole):
        if not index.isValid() or not (0 <= index.row() < self.rowCount() \
            and 0 <= index.column() < self.columnCount()):
            return QtCore.QVariant()
        row = self._dataframe.index[index.row()]
        col = self._dataframe.columns[index.column()]
        dt = self._dataframe[col].dtype

        val = self._dataframe.iloc[row][col]
        if role == QtCore.Qt.DisplayRole:
            return str(val)
        elif role == DataFrameModel.ValueRole:
            return val
        if role == DataFrameModel.DtypeRole:
            return dt
        return QtCore.QVariant()

    def roleNames(self):
        roles = {
            QtCore.Qt.DisplayRole: b'display',
            DataFrameModel.DtypeRole: b'dtype',
            DataFrameModel.ValueRole: b'value'
        }
        return roles


In [ ]:
from PyQt5 import QtCore, QtGui, QtWidgets
import pandas as pd
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(662, 512)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.horizontalLayout = QtWidgets.QHBoxLayout(self.centralwidget)
        self.horizontalLayout.setObjectName("horizontalLayout")
        self.verticalLayout = QtWidgets.QVBoxLayout()
        self.verticalLayout.setObjectName("verticalLayout")
        self.lineEdit = QtWidgets.QLineEdit(self.centralwidget)
        self.lineEdit.setObjectName("lineEdit")
        self.verticalLayout.addWidget(self.lineEdit)
        self.tableView = QtWidgets.QTableView(self.centralwidget)
        self.tableView.setObjectName("tableView")
        self.verticalLayout.addWidget(self.tableView)
        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setObjectName("pushButton")
        self.verticalLayout.addWidget(self.pushButton)
        self.horizontalLayout.addLayout(self.verticalLayout)
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 662, 21))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.pushButton.setText(_translate("MainWindow", "PushButton"))


        self.pushButton.clicked.connect(self.btn_clk)

        MainWindow.show()

    def btn_clk(self):
        path = self.lineEdit.text()
        df = pd.read_excel(path)
        df2 = df[['RANK', 'PLAYER', 'Elig. Pos.']]
        model = DataFrameModel(df2)
        self.tableView.setModel(model)
    '''
    if __name__ == "__main__":
        import sys
        app = QtWidgets.QApplication(sys.argv)
        MainWindow = QtWidgets.QMainWindow()
        ui = Ui_MainWindow()
        ui.setupUi(MainWindow)
        MainWindow.show()
        sys.exit(app.exec_())
    '''

In [ ]:
def launch_db_viewer():
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

In [ ]:
from PyQt5.QtWidgets import QTableWidget,QTableWidgetItem

self.tableWidget = QTableWidget()

# set row count
self.tableWidget.setRowCount(4)

# set column count
self.tableWidget.setColumnCount(2)

In [ ]:
import sys
from PyQt5 import QtCore, QtGui, QtWidgets

class Window(QtWidgets.QMainWindow):

    def __init__(self):
        super(Window, self).__init__()
        self.setGeometry(50, 50, 500, 300)
        self.setWindowTitle("PyQT tuts!")
        self.setWindowIcon(QtGui.QIcon('pythonlogo.png'))

        extractAction = QtWidgets.QAction("&GET TO THE CHOPPAH!!!", self)
        extractAction.setShortcut("Ctrl+Q")
        extractAction.setStatusTip('Leave The App')
        extractAction.triggered.connect(self.close_application)

        openEditor = QtWidgets.QAction("&Editor", self)
        openEditor.setShortcut("Ctrl+E")
        openEditor.setStatusTip('Open Editor')
        openEditor.triggered.connect(self.editor)

        openFile = QtWidgets.QAction("&Open File", self)
        openFile.setShortcut("Ctrl+O")
        openFile.setStatusTip('Open File')
        openFile.triggered.connect(self.file_open)

        self.statusBar()

        mainMenu = self.menuBar()
        
        fileMenu = mainMenu.addMenu('&File')
        fileMenu.addAction(extractAction)
        fileMenu.addAction(openFile)
        
        editorMenu = mainMenu.addMenu("&Editor")
        editorMenu.addAction(openEditor)

        self.home()

    def home(self):
        btn = QtWidgets.QPushButton("Quit", self)
        btn.clicked.connect(self.close_application)
        btn.resize(btn.minimumSizeHint())
        btn.move(0,100)

        extractAction = QtWidgets.QAction(QtGui.QIcon('todachoppa.png'), 'Flee the Scene', self)
        extractAction.triggered.connect(self.close_application)
        self.toolBar = self.addToolBar("Extraction")
        self.toolBar.addAction(extractAction)

        fontChoice = QtWidgets.QAction('Font', self)
        fontChoice.triggered.connect(self.font_choice)
        #self.toolBar = self.addToolBar("Font")
        self.toolBar.addAction(fontChoice)

        color = QtGui.QColor(0, 0, 0)

        fontColor = QtWidgets.QAction('Font bg Color', self)
        fontColor.triggered.connect(self.color_picker)

        self.toolBar.addAction(fontColor)

        checkBox = QtWidgets.QCheckBox('Enlarge Window', self)
        checkBox.move(300, 25)
        checkBox.stateChanged.connect(self.enlarge_window)

        self.progress = QtWidgets.QProgressBar(self)
        self.progress.setGeometry(200, 80, 250, 20)

        self.btn = QtWidgets.QPushButton("Download",self)
        self.btn.move(200,120)
        self.btn.clicked.connect(self.download)

        #print(self.style().objectName())
        self.styleChoice = QtWidgets.QLabel("Windows Vista", self)

        comboBox = QtWidgets.QComboBox(self)
        comboBox.addItem("motif")
        comboBox.addItem("Windows")
        comboBox.addItem("cde")
        comboBox.addItem("Plastique")
        comboBox.addItem("Cleanlooks")
        comboBox.addItem("windowsvista")

        comboBox.move(50, 250)
        self.styleChoice.move(50,150)
        comboBox.activated[str].connect(self.style_choice)

        cal = QtWidgets.QCalendarWidget(self)
        cal.move(500,200)
        cal.resize(200,200)

        self.show()

    def file_open(self):
        name = QtWidgets.QFileDialog.getOpenFileName(self, 'Open File')
        file = open(name,'r')

        self.editor()

        with file:
            text = file.read()
            self.textEdit.setText(text)

    def color_picker(self):
        color = QtWidgets.QColorDialog.getColor()
        self.styleChoice.setStyleSheet("QWidget { background-color: %s}" % color.name())

    def editor(self):
        self.textEdit = QtWidgets.QTextEdit()
        self.setCentralWidget(self.textEdit)


    def font_choice(self):
        font, valid = QtWidgets.QFontDialog.getFont()
        if valid:
            self.styleChoice.setFont(font)

    def style_choice(self, text):
        self.styleChoice.setText(text)
        QtWidgets.QApplication.setStyle(QtWidgets.QStyleFactory.create(text))

    def download(self):
        self.completed = 0

        while self.completed < 100:
            self.completed += 0.0001
            self.progress.setValue(self.completed)  

    def enlarge_window(self, state):
        if state == QtCore.Qt.Checked:
            self.setGeometry(50,50, 1000, 600)
        else:
            self.setGeometry(50, 50, 500, 300)
        

    def close_application(self):
        choice = QtWidgets.QMessageBox.question(self, 'Extract!',
                                            "Get into the chopper?",
                                            QtWidgets.QMessageBox.Yes | QtWidgets.QMessageBox.No)
        if choice == QtWidgets.QMessageBox.Yes:
            print("Extracting Naaaaaaoooww!!!!")
            sys.exit()
        else:
            pass
        
        

    
def run():
    app = QtWidgets.QApplication(sys.argv)
    GUI = Window()
    sys.exit(app.exec_())


run()   

In [ ]:
# Load packages.  Must have pandas and numpy.  

import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import copy
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

from GameDayFunctions.fangraphs_projection_2021 import Projection
from GameDayFunctions.draft_2021 import Draft

In [ ]:
class DraftWindow(QTableWidget):
    __init__(self, parent = None):
        super(DraftWindow,self).__init__(parent)
        

In [ ]:
autodraft_depth = 'end'
search_depth = 2
year = 2021
path_data = "projections/"
path_drafts = "Draft_Pick_Spreadsheets/"
shuffle_picks = True
ranking_method = 'FantasyPros'

First need a **Projection** object.  Projections are scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips), and include ZiPS, Steamer, and TheBat.  
In this example I create a dictionary of the three projections.  

In [ ]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS',ranking_method = ranking_method)
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer',ranking_method = ranking_method)
player_ranking_thebat = Projection(path_data=path_data,year=year,model='TheBat',ranking_method = ranking_method)
player_rankings = {"ZiPS":player_ranking_zips, "Steamer":player_ranking_steam, "TheBat":player_ranking_thebat}

Enter your roto-league-specific stats.  The default settings are shown here.  

In [ ]:
number_teams = 12
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':1}
batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] 
draft_position = 6

# Choose the projection system you prefer ()
zips_6 = Draft(player_rankings['ZiPS'], draft_position = draft_position, number_teams = number_teams,roster_spots = roster_spots,batter_stats = batter_stats,pitcher_stats = pitcher_stats)
steamer_6 = Draft(player_rankings['Steamer'], draft_position = draft_position, number_teams = number_teams,roster_spots = roster_spots,batter_stats = batter_stats,pitcher_stats = pitcher_stats)
thebat_6 = Draft(player_rankings['TheBat'], draft_position = draft_position, number_teams = number_teams,roster_spots = roster_spots,batter_stats = batter_stats,pitcher_stats = pitcher_stats)

To use while drafting in real-time, simply record the draft results in a spreadsheet (.xlsx, so if you're using Numbers, need to export to Excel) and run the cell. 

Examples below show results for after 2, 3, and 4 rounds.  Notice that at each stage the projected placement in the standings improves.  

In [ ]:
# Example pick in the 1st round

draft_pick_file = 'MockDraftFP_Round_1.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 2, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 1st round

draft_pick_file = 'MockDraftFP_Round_1.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 2, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 2nd round
draft_pick_file = 'MockDraftFP_Round_2.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 2, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 2nd round
draft_pick_file = 'MockDraftFP_Round_2.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 3, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 3rd round

draft_pick_file = 'MockDraftFP_Round_3.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 3, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 3rd round

draft_pick_file = 'MockDraftFP_Round_3.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 3, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 4nd round

draft_pick_file = 'MockDraftFP_Round_4.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 4nd round

draft_pick_file = 'MockDraftFP_Round_4.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 5th round

draft_pick_file = 'MockDraftFP_Round_5.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 5th round

draft_pick_file = 'MockDraftFP_Round_5.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 5th round

draft_pick_file = 'MockDraftFP_Round_5.xlsx'
player_to_pick, projected_roto_stats = thebat_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 6th round

draft_pick_file = 'MockDraftFP_Round_6.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 6th round

draft_pick_file = 'MockDraftFP_Round_6.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 6th round

draft_pick_file = 'MockDraftFP_Round_6.xlsx'
player_to_pick, projected_roto_stats = thebat_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 7th round

draft_pick_file = 'MockDraftFP_Round_7.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 7th round

draft_pick_file = 'MockDraftFP_Round_7.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 7th round

draft_pick_file = 'MockDraftFP_Round_7.xlsx'
player_to_pick, projected_roto_stats = thebat_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 8th round

draft_pick_file = 'MockDraftFP_Round_8.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 8th round

draft_pick_file = 'MockDraftFP_Round_8.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 9th round

draft_pick_file = 'MockDraftFP_Round_9.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 9th round

draft_pick_file = 'MockDraftFP_Round_9.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 9th round

draft_pick_file = 'MockDraftFP_Round_9.xlsx'
player_to_pick, projected_roto_stats = thebat_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 10th round

draft_pick_file = 'MockDraftFP_Round_10.xlsx'
player_to_pick, projected_roto_stats = zips_16.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 11th round

draft_pick_file = 'MockDraftFP_Round_11.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 12th round

draft_pick_file = 'MockDraftFP_Round_12.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 13th round

draft_pick_file = 'MockDraftFP_Round_13.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 14th round

draft_pick_file = 'MockDraftFP_Round_14.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 15th round

draft_pick_file = 'MockDraftFP_Round_15.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Example pick in the 16th round

draft_pick_file = 'MockDraftFP_Round_16.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [ ]:
# Projected_roto_stats contains the estimated results from all teams given the recommented pick

projected_stats = projected_roto_stats[0]
projected_standings = projected_roto_stats[3]
projected_place = projected_roto_stats[4]
projected_scores = projected_roto_stats[5]

In [ ]:
# The projected_stats give the raw numbers

projected_stats

In [ ]:
# And projected_scores show how they translate to roto scores

projected_scores

In [ ]:
# Finally, the projected_standings shows the resulting standings

projected_standings

In [ ]:
# But if you just want to know how your team finishes, that's projected_place

projected_place